## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-22-59-41 +0000,nypost,Democrat candidate running against NY Rep. Mik...,https://nypost.com/2026/02/11/us-news/democrat...
1,2026-02-11-22-56-35 +0000,nyt,Canada Updates: Police Identify Suspect in Mas...,https://www.nytimes.com/live/2026/02/11/world/...
2,2026-02-11-22-56-33 +0000,nyt,"In Tumbler Ridge, a Community in Stunned Agony",https://www.nytimes.com/live/2026/02/11/world/...
3,2026-02-11-22-55-33 +0000,nypost,Ford CEO admits that ‘the customer has spoken’...,https://nypost.com/2026/02/11/business/ford-ce...
4,2026-02-11-22-53-18 +0000,nypost,"Two teens stabbed outside NYC middle school, s...",https://nypost.com/2026/02/11/us-news/two-teen...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2466/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
134,trump,36
11,canada,18
162,epstein,18
112,new,16
17,shooting,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...,118
333,2026-02-11-01-14-19 +0000,latimes,L.A. County officials push new sales tax to of...,https://www.latimes.com/california/story/2026-...,90
246,2026-02-11-11-57-00 +0000,wsj,Commerce Secretary Howard Lutnick became the f...,https://www.wsj.com/politics/policy/lutnicks-f...,87
349,2026-02-10-23-53-00 +0000,wsj,Canada’s Carney Tries to Resolve Trump Demands...,https://www.wsj.com/world/americas/carney-look...,79
112,2026-02-11-20-02-56 +0000,startribune,New Trump administration rule will bar green-c...,https://www.startribune.com/sba-minnesota-rule...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,118,2026-02-11-15-19-00 +0000,wsj,House lawmakers rejected an attempt by Speaker...,https://www.wsj.com/politics/policy/house-reje...
290,77,2026-02-11-09-03-03 +0000,nypost,‘Gunperson’ in a ‘dress’ behind Canada’s deadl...,https://nypost.com/2026/02/11/world-news/tumbl...
170,69,2026-02-11-17-27-07 +0000,nypost,Ex-Google CEO Eric Schmidt spent New Year’s wi...,https://nypost.com/2026/02/11/business/ex-goog...
62,62,2026-02-11-21-42-00 +0000,wsj,"At a congressional hearing, Attorney General P...",https://www.wsj.com/politics/policy/pam-bondi-...
72,44,2026-02-11-21-21-01 +0000,nypost,Zohran Mamdani spews head-scratching reason wh...,https://nypost.com/2026/02/11/us-news/zohran-m...
140,44,2026-02-11-18-56-22 +0000,latimes,Warner Bros. Discovery board faces pressure as...,https://www.latimes.com/entertainment-arts/bus...
124,40,2026-02-11-19-35-00 +0000,wsj,LA28 Olympics Organizers Say Wasserman Will Re...,https://www.wsj.com/us-news/casey-wasserman-28...
74,39,2026-02-11-21-18-29 +0000,nypost,Investigators find black glove during hunt for...,https://nypost.com/2026/02/11/us-news/investig...
333,38,2026-02-11-01-14-19 +0000,latimes,L.A. County officials push new sales tax to of...,https://www.latimes.com/california/story/2026-...
66,33,2026-02-11-21-36-27 +0000,nypost,Two sisters dead in suspicious Hollywood fire ...,https://nypost.com/2026/02/11/us-news/two-sist...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
